In [ ]:
import numpy as np
import pandas as pd
import scanpy as sc
import pymn
import anndata as ad
import time
import os
from pyprojroot import here
import resource

In [ ]:
#load, then run cpm

In [ ]:
here()
base_data_folder = "/vault/lfrench/mouse_brain_cluster_replicability/data/"

In [ ]:
## Won't run on Dactyl
adata_Zeng = sc.read_h5ad(base_data_folder + "/whole_mouse_brain/processed/zeng/subsets/AIT21.0.merged.with_multiome.h5ad")


In [ ]:
Zeng_cluster_info_AWS = pd.read_csv(base_data_folder + "/whole_mouse_brain/zeng/from_aws/AIT21.0/AIT21_annotation_freeze_081523.tsv", sep = '\t')
#set study ID here to prevent view as actual memory grabs 
Zeng_cluster_info_AWS['study_id'] = "Zeng"
#set it to be strings
adata_Zeng.obs['cl'] = adata_Zeng.obs['cl'].astype(str)
Zeng_cluster_info_AWS['cl'] = Zeng_cluster_info_AWS['cl'].astype(str)
adata_Zeng.obs.index = adata_Zeng.obs.index.astype(str)
adata_Zeng.obs = adata_Zeng.obs.merge(Zeng_cluster_info_AWS, how="left", on='cl')
#use ensmbl ID as index
adata_Zeng.var = adata_Zeng.var.set_index("gene_identifier")
#LQ or low quality cells
sum(adata_Zeng.obs['subclass_label'] == "LQ")
#remove #LQ
adata_Zeng.obs.index = adata_Zeng.obs.index.astype(str)
adata_Zeng.obs["filter_out"] = adata_Zeng.obs['subclass_label'] == "LQ"
adata_Zeng = adata_Zeng[~adata_Zeng.obs["filter_out"],:]
adata_Zeng.obs.index = adata_Zeng.obs.index.astype(str)


In [ ]:
print(adata_Zeng)

In [ ]:
adata_Zeng.var

In [ ]:
sc.pp.normalize_total(adata_Zeng, target_sum=1e6, inplace=True)

In [ ]:
# Split the index into parts
index_parts = np.array_split(adata_Zeng.var.index, 40)

# write out the parts
for i, part in enumerate(index_parts):
    print(str(i))
    adata_Zeng[:, part].write(base_data_folder + "/whole_mouse_brain/processed/zeng/subsets/AIT21.0.merged.with_multiome.cpm.part_"+str(i) + ".h5ad")

In [ ]:
base_data_folder

In [ ]:
############Macosko - code duplication

In [ ]:
adata_Macosko = sc.read_h5ad(base_data_folder + "/whole_mouse_brain/macosko/from_google_drive/Macosko_Mouse_Atlas_Single_Nuclei.Use_Backed.h5ad")
sc.pp.normalize_total(adata_Macosko, target_sum=1e6, inplace=True)


In [ ]:
adata_Macosko.obs['study_id'] = "Macosko"
adata_Macosko.obs.index = adata_Macosko.obs.index.to_numpy(dtype="str")

In [ ]:
# Split the index into parts

index_parts = np.array_split(adata_Macosko.var.index, 20)

# write out the parts
for i, part in enumerate(index_parts):
    print(str(i))
    adata_Macosko[:, part].write(base_data_folder + "/whole_mouse_brain/processed/macosko/subsets/Macosko_Mouse_Atlas_Single_Nuclei.Use_Backed.cpm.part_"+str(i) + ".h5ad")

In [ ]:
print("Done!")